In [6]:
import pandas as pd
import numpy as np

In [3]:
comp = pd.read_csv("/Users/sean/Documents/school/phd/research/warm_starting/vws/experiments/outputs/miplib_5000_gurobi/unperturbed_bound_table.csv")
comp

,instance,terms,lpBound New,disjunctiveDualBound New,lpBoundPostVpc New,rootDualBound None,rootDualBound New,my ratio
0,1,4,6637.188027,6641.585617,6640.550919,6714.074468,6716.532958,1.000366
1,1,16,6637.188027,6644.289771,6642.855642,6714.074468,6716.497950,1.000361
2,1,64,6637.188027,6647.648085,6641.628977,6714.074468,6708.438499,0.999161
3,23588,4,7649.866134,7799.912563,7774.120220,7770.367871,7797.326487,1.003469
4,23588,16,7649.866134,7867.860624,7862.267452,7770.367871,7867.450026,1.012494
...,...,...,...,...,...,...,...,...
198,timtab1CUTS,16,618783.684884,624954.797154,620366.145397,632895.730479,633658.012677,1.001204
199,timtab1CUTS,64,618783.684884,632296.230889,630413.602939,632895.730479,633482.153061,1.000927
200,tr12-30,4,25302.209524,25611.359565,25498.481331,129146.721533,129090.840776,0.999567
201,tr12-30,16,25302.209524,26546.641734,25805.491566,129146.721533,129090.840776,0.999567


In [4]:
groups = ["instance", "terms"]
fields = ["lpBound New", "disjunctiveDualBound New", "lpBoundPostVpc New", "rootDualBound None", "rootDualBound New"]

adf = pd.read_csv("aleks_results.csv")
adf = adf[["INSTANCE", "disj_terms", "LP OBJ", "BEST DISJ OBJ", "VPC OBJ", "AVG REF LAST_CUT_PASS", "AVG REF+V LAST_CUT_PASS"]]
adf.columns = groups + fields
adf["aleks ratio"] = adf["rootDualBound New"] / adf["rootDualBound None"]
adf['instance'] = adf['instance'].str.replace('_presolved', '')
adf

,instance,terms,lpBound New,disjunctiveDualBound New,lpBoundPostVpc New,rootDualBound None,rootDualBound New,aleks ratio
0,22433,2.0,2.124053e+04,21314.98074,2.131206e+04,2.141369e+04,2.140756e+04,0.999713
1,23588,2.0,7.649866e+03,7746.197749,7.726713e+03,7.757177e+03,7.789200e+03,1.004128
2,10teams,2.0,9.170000e+02,917,9.170000e+02,9.240000e+02,9.240000e+02,1.000000
3,30n20b8,2.0,1.241230e+02,124.2095421,1.241823e+02,1.542021e+02,1.528972e+02,0.991538
4,50v-10,2.0,2.879066e+03,2884.065687,2.884066e+03,3.195241e+03,3.199688e+03,1.001392
...,...,...,...,...,...,...,...,...
2637,vpm1,64.0,1.643333e+01,16.71111111,1.660000e+01,1.886429e+01,1.900595e+01,1.007510
2638,vpm2,64.0,1.113563e+01,11.5093094,1.134045e+01,1.305095e+01,1.305186e+01,1.000070
2639,wachplan,64.0,-9.000000e+00,-9,NaN,0.000000e+00,0.000000e+00,NaN
2640,zib54-UUE,64.0,4.330388e+06,5389605.005,4.850439e+06,8.432754e+06,8.397464e+06,0.995815


In [7]:
# join comp and adf on instance and terms
diff = comp[groups + ["my ratio"]].merge(adf[groups + ["aleks ratio"]], on=groups)
diff[np.abs(diff["my ratio"] - diff["aleks ratio"]) > .02]

,instance,terms,my ratio,aleks ratio
21,bienst2,4,1.169338,1.006596
22,bienst2,16,1.086287,1.195931
28,breastcancer-regularized,4,1.280701,1.042262
29,breastcancer-regularized,16,1.279734,0.925203
31,cov1075,16,1.000000,1.034961
60,gsvm2rl3,4,1.509963,1.065242
62,gsvm2rl3,64,1.267143,1.099603
91,misc07,64,1.036042,1.003950
113,neos-4333464-siret,16,1.040162,0.982945
114,neos-4333464-siret,64,1.020563,0.988091


In [8]:
diff[diff.columns[1:]].groupby("terms").mean()

,my ratio,aleks ratio
terms,,
4,1.011423,1.002317
16,1.007624,1.004213
64,1.008820,1.003497


In [9]:
adf[["terms", "aleks ratio"]].groupby("terms").mean()

,aleks ratio
terms,
2.0,1.227926
4.0,1.218641
8.0,1.231829
16.0,1.240385
32.0,1.263117
64.0,1.034405
